# Create a notebook to perform Runge-Kutte integration for multiple coupled variables

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define our coupled variables to integrate

In [4]:
def dydx(x,y):
    
    # set the derivatives
    
    # our equation is d^2y/dx^2 = -y
    
    # so we can write
    # dy/dx = z
    # dz/dx = -y
    
    # we will set y = y[0]
    # we will set z = y[1]
    
    # declare an array
    y_derivs = np.zeros(2)
    
    #set dy/dx = z
    y_derivs[0] = y[1]
    
    # set dz/dx = -y
    y_derivs[1] = -1*y[0]
    
    # we return the arrays of dy/dx
    return y_derivs

## Define the 4th order RK method

In [5]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    # declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    # define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    # define x at 1 step
    x_ipo = xi + h
    
    # declare a temp array
    y_temp = np.zeros(nv)
    
    # get our k1 value
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    # get our k2 value
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    # get our k3 value
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    # get our k4 value
    y_temp[:] = y[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    # advance y by a step of h
    yipo = yi +(k1 + 2*k2 + 2*k3 +k4)/6
    
    # This is an array
    return yipo

## Define an adaptive step size driver for RK4

In [6]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    # define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    # set a maximum number of iterations
    imax= 10000
    
    # set an iteraton variable
    i =0
    
    # create an error
    Delta = np.full(nv, 2*tol)
    
    # remember the step that we were asked to take
    h_step = h
    
    # adjust the step
    while(Delta.max()/tol>1.0):
        
        # estimate our error by taking one step of size h
        # versus two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        # compute an error
        Delta = np.fabs(y_2 - y_11)
        
        # If the error is too large, take a smaller step
        if(Delta.max/tol>1.0):
            
            # our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        # check iterations
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad")
            raise StopIteration("Ending after i = ",i)
            
        # iterate
        i += 1
        
    # next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    # return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    # dydx is the derivative wrt x
    # [a,b] are the lower and upper bounds
    # y_a are the boundary conditions
    # tol is the tolerance for integrating y
    
    # define our starting step
    xi = a
    yi = y_a.copy()
    
    # an initial step size == make very small
    h = 1.0e-4*(b-a)
    
    # set a maximum iteration
    imax = 10000
    
    # set an iteration variable
    i = 0
    
    # set the number of coupled of ODEs to the size of y_a
    nv = len(y_a)
    
    # set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    # set a flag
    flag = 1
    
    # loop until we reach the right side
    while(flag):
        
        # calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,,nv,h,tol)
        
        
    